**Name:** SIVAKUMAR Srinivas

**EID:** ssivakuma2

**Kaggle Team Name:** Srinivas Sivakumar

# CS4487 - Assignment 2 - Sound Effects Retrieval
Due date: November 5, 2018, 11:59pm

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import csv
from scipy import io
import pickle
from IPython.display import Audio, display

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
D:\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
def showAudio(info):
    display(Audio(info['previews']['preview-lq-mp3']))

def load_pickle(fname):
    f = open(fname, 'rb')
    out = pickle.load(f)
    f.close()
    return out

In [3]:
train_tags = load_pickle('sounds-data/train_tags.pickle3')
train_mfccs = load_pickle('sounds-data/train_mfccs.pickle3')
train_info = load_pickle('sounds-data/train_info.pickle3')

valid_mfccs = load_pickle('sounds-data/val_mfccs.pickle3')
valid_info = load_pickle('sounds-data/val_info.pickle3')
valid_matches = load_pickle('sounds-data/val_matches.pickle3')

test_mfccs = load_pickle('sounds-data/test_mfccs.pickle3')
test_info = load_pickle('sounds-data/test_info.pickle3')

In [4]:
def nn_retr(testX, trainX):
    # do retrieval using nearest neighbors
    # for each entry in testX, find the top 10 closest entries in trainX using Euclidean distance
    
    test_retr = []  # retrieval list

    # for each test document vector
    for tX in testX:
        # calculate the distance between tX and all training document vectors
        # (using Euclidean distance)
        if tX.ndim == 1:
            ttX = tX.reshape((1,tX.shape[0]))
        else:
            ttX = tX;
        D = metrics.pairwise_distances(ttX, trainX, metric='euclidean')
        Df = D.flatten()
        
        # find the 10 with smallest distance
        # (faster: bestmatches are the top-10, but not sorted yet)
        bestmatches_unsorted = argpartition(Df, 10)[0:10]
                
        # now sort bestmatches
        bestmatches_tmp = argsort(Df[bestmatches_unsorted])
        bestmatches = bestmatches_unsorted[bestmatches_tmp]        
        
        # slow, since it sorts everything
        #bestmatches = argsort(D.flatten())[0:10]
    
        # add to retrieval list
        test_retr.append(bestmatches)

    return test_retr

In [5]:
def calc_MAP(Ytrue, Ypred, K=10):
    # calculate the mean average precision
    # returns the MAP for the whole set, and the AP for each document
    
    AP = zeros(len(Ytrue))
    # for each document
    for j in range(len(Ytrue)):
        Yt = Ytrue[j]
        if not isinstance(Yt, ndarray):
            Yt = array([Yt])
        Yp = Ypred[j]
        # calculate average precision @ K
        nummatches = 0.
        score = 0.
        for i,y in enumerate(Yp[0:K]):
            # if match at this level, calculate P
            if any(y==Yt):
                nummatches += 1.
                P = nummatches / (i+1)
                score += P
        AP[j] = score / K #min(K, len(Yt))

    # mean over all documents
    MAP = mean(AP)
    return (MAP, AP)

# write a kaggle submission file for retrieval
def write_csv_kaggle_retr_sub(fname, Yretr):
    # header
    tmp = [['Id', 'RetrievedDocuments']]
    
    # add ID numbers for each Y, and usage if necessary
    for i in range(len(Yretr)):
        y = Yretr[i]
        ystr = [str(q) for q in y]
        tmp2 = [str(i+1)]
        tmp2.append(" ".join(ystr))        
        tmp.append(tmp2)
        
    # write CSV file
    f = open(fname, 'w')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

In [6]:
# compute delta MFCCs
def compute_delta_mfccs(mfccs):
    dmfccs = []
    for m in mfccs:
        tmp = m[1:] - m[0:-1]
        dm = hstack((m[0:-1], tmp))
        dmfccs.append(dm)
    return dmfccs

In [7]:
train_dmfccs = compute_delta_mfccs(train_mfccs)
test_dmfccs  = compute_delta_mfccs(test_mfccs)
valid_dmfccs = compute_delta_mfccs(valid_mfccs)
all_dmfccs = vstack(train_dmfccs)

In [8]:
def bow_transform(model, mfccs):
    numwords = model.means_.shape[0]
    bows = zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

In [9]:
def nn_retr(testX, trainX):
    # do retrieval using nearest neighbors
    # for each entry in testX, find the top 10 closest entries in trainX using Manhattan distance
    
    test_retr = []  # retrieval list

    # for each test document vector
    for tX in testX:
        # calculate the distance between tX and all training document vectors
        # (using Euclidean distance)
        if tX.ndim == 1:
            ttX = tX.reshape((1,tX.shape[0]))
        else:
            ttX = tX;
        D = metrics.pairwise_distances(ttX, trainX, metric='cityblock')
        Df = D.flatten()
        
        # find the 10 with smallest distance
        # (faster: bestmatches are the top-10, but not sorted yet)
        bestmatches_unsorted = argpartition(Df, 10)[0:10]
                
        # now sort bestmatches
        bestmatches_tmp = argsort(Df[bestmatches_unsorted])
        bestmatches = bestmatches_unsorted[bestmatches_tmp]        
        
        # slow, since it sorts everything
        #bestmatches = argsort(D.flatten())[0:10]
    
        # add to retrieval list
        test_retr.append(bestmatches)

    return test_retr

In [10]:
def pcaReduction(features,components):
    pca = decomposition.PCA(n_components=components)
    result = pca.fit_transform(features)
    return (pca,result)

In [11]:
pca,new_all_dmfccs = pcaReduction(all_dmfccs,8)
new_train_dmfccs = []
for j in range(len(train_dmfccs)):
    new_train_dmfccs.append(pca.transform(train_dmfccs[j]))
        
new_valid_dmfccs = []
for j in range(len(valid_dmfccs)):
    new_valid_dmfccs.append(pca.transform(valid_dmfccs[j]))
    
new_test_dmfccs=[]
for j in range(len(test_dmfccs)):
        new_test_dmfccs.append(pca.transform(test_dmfccs[j]))


gmm_diag = mixture.GaussianMixture(n_components=500, covariance_type='diag',
                               random_state=4487, verbose=2)
gmm_diag.fit(new_all_dmfccs[0::10])


Initialization 0
  Iteration 0	 time lapse 56.20184s	 ll change inf
  Iteration 10	 time lapse 28.43504s	 ll change 0.01825
  Iteration 20	 time lapse 29.54881s	 ll change 0.00669
  Iteration 30	 time lapse 30.60758s	 ll change 0.00543
  Iteration 40	 time lapse 31.03956s	 ll change 0.00197
  Iteration 50	 time lapse 31.53767s	 ll change 0.00175
  Iteration 60	 time lapse 33.83652s	 ll change 0.00192
  Iteration 70	 time lapse 33.25508s	 ll change 0.00116
Initialization converged: True	 time lapse 300.19637s	 ll -35.59491


GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=500, n_init=1, precisions_init=None,
        random_state=4487, reg_covar=1e-06, tol=0.001, verbose=2,
        verbose_interval=10, warm_start=False, weights_init=None)

In [13]:
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
train_bow = bow_transform(gmm_diag, new_train_dmfccs)
valid_bow = bow_transform(gmm_diag, new_valid_dmfccs)    
train_Xtf = tf_trans.fit_transform(train_bow)
valid_Xtf = tf_trans.transform(valid_bow)
test_bow = bow_transform(gmm_diag, new_test_dmfccs)
test_Xtf = tf_trans.transform(test_bow)

val_retr  = nn_retr(valid_Xtf, train_Xtf)
vMAP,vAP = calc_MAP(valid_matches, val_retr)
print("GMM using diagonal covariance and Manhattan distance, MAP =",vMAP)

test_retr = nn_retr(test_Xtf, train_Xtf)
write_csv_kaggle_retr_sub("final_submission.csv", test_retr)

GMM using diagonal covariance and Manhattan distance, MAP = 0.15688126059485286
